In [1]:
import pandas as pd
import os
import yfinance as yf
from yahooquery import Ticker
from yahooquery import Screener
from datetime import datetime
s = Screener()

data_path = r'C:\Users\pythonProject\data\data_produced\yquerry_screens\options'
folder_path = r"C:\Users\pythonProject\data\data_produced\yquerry_screens"
today = datetime.today().strftime('%Y%m%d')
file_name = data_path + f"\\{today}_option_data_multiple_tickers.csv"
scr_file_name = data_path + f"\\{today}_option_screen_base_data.xlsx"
excel_file_path = os.path.join(data_path, scr_file_name) 
screen_option_data_file = data_path + f"\\{today}_screen_option_data_file.csv"


In [2]:
stock = 'FSR'
file_name_temp = data_path + f"\\{today}_{stock}_option_data.csv"
t = Ticker(stock, asynchronous=True)
df = t.option_chain
df.to_csv(file_name_temp)

In [3]:
##### Creating file ==> {today}_option_screen_base_data.xlsx
"""
This program reads the screen_list and generates the data for these screens using yahoo API.Then writes
it to an excel file with tab for each screen.

Additionally , all the data is also combined into a single dataframe combined_df for further use .

"""
screen_list = ['aggressive_small_caps', 'most_watched_tickers', 'most_actives', 'upside_breakout_stocks_daily',
               'top_options_open_interest', 'most_shorted_stocks']

dict_type = {'key1' : 'quotes', 'key2' : 'symbol'}

columns = ['symbol','customPriceAlertConfidence','averageAnalystRating','regularMarketPrice','fiftyTwoWeekRange','sharesOutstanding','fiftyTwoWeekLow','fiftyTwoWeekHigh',
           'regularMarketVolume',  'averageDailyVolume10Day', 'averageDailyVolume3Month', 
           'marketCap', 'bookValue', 'longName']

columns_most_oi = ['symbol', 'optionsType', 'strike', 'expireDate', 'openInterest', 'regularMarketPrice', 'fiftyTwoWeekRange', 'regularMarketVolume',
           'priceHint']

# Function to convert Unix timestamp to formatted date
def convert_unix_to_date(timestamp):
    date_time = datetime.utcfromtimestamp(timestamp)
    formatted_date = date_time.strftime('%Y-%m-%d')
    return formatted_date

def screen_tickers(scr, columns):
    screen_data_dict = s.get_screeners([scr])
    screen_data_list = screen_data_dict[scr]['quotes']
    return pd.DataFrame(screen_data_list, columns = columns)

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    combined_df = None
    for scr in screen_list: 
        screen_df = screen_tickers(scr, columns)
        # Append subsequent DataFrames to the combined DataFrame (in-place)
        if combined_df is None:
            combined_df = screen_df  # First iteration: assign screen_df to combined_df
        else:
            combined_df = pd.concat([combined_df, screen_df], ignore_index=True)  # Append subsequent DataFrames
        screen_df.to_excel(writer, sheet_name=scr, index=True)



In [4]:
"""
this program creates a list of uniques symbols from the screen data extracted previously and then uses
this new list to extraxt option data for each symbol.

"""
stocklist = combined_df['symbol'].drop_duplicates().tolist()
print(f"there are {len(stocklist)} stocks in this list")

def fetch_option_data(stocklist):
    t = Ticker(stocklist, asynchronous=True)
    most_option_df = t.option_chain
    
    return most_option_df.sort_values(by="symbol")

most_option_df = fetch_option_data(stocklist)
most_option_df.to_csv(screen_option_data_file) 
most_option_df.info()

there are 120 stocks in this list
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 58385 entries, ('AAPL', Timestamp('2024-03-01 00:00:00'), 'calls') to ('XOM', Timestamp('2026-12-18 00:00:00'), 'puts')
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   contractSymbol     58385 non-null  object        
 1   strike             58385 non-null  float64       
 2   currency           58385 non-null  object        
 3   lastPrice          58385 non-null  float64       
 4   change             58385 non-null  float64       
 5   percentChange      58385 non-null  float64       
 6   volume             58385 non-null  float64       
 7   openInterest       58385 non-null  float64       
 8   bid                58385 non-null  float64       
 9   ask                58385 non-null  float64       
 10  contractSize       58385 non-null  object        
 11  lastTradeDate      58385 non-null  datetime

In [5]:
"""
next , fetch latest prices for the stocks in the screen data list

"""

def fetch_latest_prices(tickers):
    error_list = []
    stock_data = pd.DataFrame()  # Initialize an empty DataFrame

    for ticker in tickers:
        try:
            data = yf.download(ticker, period="1d", interval="1m")
            latest_price = data["Close"].iloc[-1]
            stock_data = pd.concat([stock_data, pd.DataFrame({"Ticker": ticker, "Latest_Price": latest_price}, index=[0])])
        except Exception as e:
            print(f"Error fetching price for {ticker}: {e}")
            error_list.append(ticker)
    
    stock_data = stock_data.sort_values(by="Ticker")
    return stock_data, error_list

# Example usage:
#tickers = ["AAPL", "GOOG", "MSFT", "INVALID"]  # Replace with your list of tickers
stock_data, error_list = fetch_latest_prices(stocklist)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******


1 Failed download:
['VIX240522C00060000']: Exception('%ticker%: No price data found, symbol may be delisted (period=1d)')



Error fetching price for VIX240522C00060000: single positional indexer is out-of-bounds
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['VIX240320C00047500']: Exception('%ticker%: No price data found, symbol may be delisted (period=1d)')



Error fetching price for VIX240320C00047500: single positional indexer is out-of-bounds
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['EEM250117C00040000']: Exception('%ticker%: No price data found, symbol may be delisted (period=1d)')



Error fetching price for EEM250117C00040000: single positional indexer is out-of-bounds
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*************

In [6]:
## combined_df ,screen_df ,most_option_df, stock_data
## print(combined_df.info())
## print(screen_df.info())
## print(most_option_df.info())
##stock_data.set_index("Ticker", inplace = True)
"""
This program calculates potential returns for all the options and then ranks based on the returs calculated. User
sets a future price against which these returns are calculated to help make a descision on risk/reward
"""
import numpy as np  # Import numpy for NaN handling

temp_file = data_path + f"\\temp_file.csv"

joined_df = most_option_df.join(stock_data, how="outer")
# Apply the conditional logic using np.where
joined_df["riskPremium"] = np.where(joined_df["inTheMoney"] ,
                                    (joined_df["strike"] + joined_df["lastPrice"]) - joined_df["Latest_Price"],
                                    joined_df["lastPrice"])

joined_df.to_csv(temp_file)
#calculate_future_stock_prices(joined_df, stocklist )




ValueError: cannot join with no overlapping index names

In [113]:
print(stock_data)


                    Latest_Price
Ticker                          
AAPL                  195.470001
ABR                    13.865000
AG                      4.810100
AHT                     1.475000
AI                     26.150000
...                          ...
VIX240417C00060000      0.130000
VZ                     41.955002
WMT                   162.455002
XLE240315P00080000      1.590000
XOM                    99.095001

[101 rows x 1 columns]


In [7]:

stocklist =['FSR','RIVN','TSLA','DIS','GM','HD','BABA','AAPL','APPS','PLTR',
            'EXPR','MARA','GME','BB','RKT','HD','SCHW','NIO']
#stocklist =['FSR']
t = Ticker(stocklist, asynchronous=True)
df = t.option_chain
df.to_csv(file_name)

In [4]:
def optionratio():
    
    df_cols = ['symbol', 'expiration', 'optionType', 'strike','lastPrice', 'volume','openInterest']
    group_cols = ['symbol','strike', 'expiration','optionType']
    agg_dict = {'openInterest' : 'sum', 'volume' : 'sum'}
    oi_df = pd.read_csv(file_name)
    oi_df = oi_df.filter(df_cols)
    print(oi_df.head())
    oi_df = oi_df.groupby(group_cols).agg(agg_dict)
    oi_df = oi_df.unstack(level = 3)
    oi_df.to_csv(file_name_oi)
    return oi_df

df_temp = optionratio()


  symbol  expiration optionType  strike  lastPrice  volume  openInterest
0   AAPL  2024-01-26      calls    60.0     131.40     2.0             0
1   AAPL  2024-01-26      calls    90.0      93.45     0.0             0
2   AAPL  2024-01-26      calls   100.0      87.92     7.0             7
3   AAPL  2024-01-26      calls   110.0      73.75     0.0             1
4   AAPL  2024-01-26      calls   125.0      65.51     2.0             3
